In [139]:
import os
import json
from speach import elan
import string
import pandas as pd
import warnings

In [85]:
def parsefile(file):
    table = pd.read_csv(file, sep='\t')
    speakers = {i.split('@')[-1] for i in table.columns}
    glossing = []
    for speaker in speakers:
        table_A = table[[f'tx@{speaker}', f'mb@{speaker}', f'ge@{speaker}']]
        table_A.dropna(inplace=True)
        table_A = table_A.groupby(f'tx@{speaker}').aggregate({f'mb@{speaker}': list, f'ge@{speaker}': list})
        for row in table_A.itertuples():
            morph = 0
            for word in row[0].split():
                wordform = word.lower().strip(punct)
                glosses = []
                root = False
                if morph >= len(row[2]):
                    continue
                while morph < len(row[2]) and not(root == True and row[2][morph][0] not in ('-', '=')):
                    if row[2][morph][-1] in ('-', '='):
                        glosses.append(row[2][morph])
                    if row[2][morph][-1] not in ('-', '=') and row[2][morph][0] not in ('-', '='):
                        root = True
                        glosses.append(row[2][morph])
                    if row[2][morph][0] in ('-', '='):
                        glosses.append(row[2][morph])
                    morph += 1
                glossing.append((wordform, set(glosses)))
    return glossing

In [88]:
warnings.simplefilter('ignore')
glossing = []
for file in os.listdir('ELAN'):
    if file.split('.')[-1] == 'txt':
        glossing.extend(parsefile(f'ELAN/{file}'))        

In [110]:
def convarg(text):
    if text[:2].isnumeric():
        cl = 2
    else:
        cl = 1
    
    new = text[:cl]
    
    if text[cl].islower():
        new += text[cl:cl+2] + '.'
    
    if text[-2] == 'O':
        new += 'obj'
    elif text[-2] == 'S':
        new += 'sbj'
    
    return new

In [142]:
standard = []

In [143]:
for wordform in glossing:
    form = wordform[0]
    glosses = wordform[1]
    new_glosses = []
    for gloss in glosses:
        if gloss[0].isnumeric() and gloss[-2:] in ('O-', 'S-'):
            new_glosses.extend([convarg(gloss)])
            continue
        if len(gloss.split('.')) > 1 and gloss.split('.')[0].isnumeric and not gloss.split('.')[1].isupper():
            new_glosses.extend([gloss.split('.')[0]])
            continue
        if gloss[-4:] == 'POSS':
            new_glosses.extend([gloss.replace('POSS', ''), 'assoc'])
            continue
        if gloss.replace('_', '').isalpha() and not gloss.isupper():
            continue
        if gloss == 'AUX':
            new_glosses.extend(['cop'])
            continue
        new_glosses.extend([gloss.lower().strip('-=')])
    entry = (form, new_glosses)
    if entry not in standard:
        standard.append(entry)

In [145]:
with open('standard.json', 'w', encoding='utf-8') as f:
    json.dump(standard, f, ensure_ascii=False, indent='\t')